![Neptune + fastai](https://neptune.ai/wp-content/uploads/2023/09/fastai.svg)

# Neptune + fastai

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/fastai/notebooks/Neptune_fastai.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/fastai/notebooks/Neptune_fastai.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/fastai-integration/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=fastai-dashboard-1f456716-f509-4432-b8b3-a7f5242703b6&shortId=FAS-1895&type=run"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/fastai/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Log basic metadata using `NeptuneCallback()`,
* Do single and multi phase logging using `NeptuneCallback()`,
* Log model weights using `NeptuneCallback()`,
* Log images to a Neptune `run` using the Neptune client library (`neptune`).

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
! pip install -U neptune neptune-fastai fastai torch torchvision numpy

## Basic Example

**Import libraries**

In [ ]:
import neptune
import torch
from fastai.callback.all import SaveModelCallback
from fastai.vision.all import (
    ImageDataLoaders,
    URLs,
    accuracy,
    resnet18,
    untar_data,
    vision_learner,
)
from neptune.integrations.fastai import NeptuneCallback
from neptune.types import File

### Create a Neptune run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
run = neptune.init_run(
    project="common/fastai-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags="notebook-run",
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

**Dataset**

In [ ]:
path = untar_data(URLs.MNIST_TINY)
dls = ImageDataLoaders.from_csv(path, num_workers=0)

In [ ]:
dls.show_batch()

### Log metadata using `NeptuneCallback()`
By using `NeptuneCallback()`, the following is automatically logged to Neptune for you:
- Hyperparameters
- Loss
- Metrics
- Best model weights 
- Model architecture

In [ ]:
learn = vision_learner(
    dls,
    resnet18,
    cbs=[SaveModelCallback(), NeptuneCallback(run=run, base_namespace="experiment")],
)

In [ ]:
learn.fit_one_cycle(1)

## More options

### Single and Multi phase logging

#### Log on a single training phase

In [ ]:
learn = vision_learner(dls, resnet18, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(1, cbs=[NeptuneCallback(run=run, base_namespace="experiment_1")])

#### Log all training phases of the learner

In [ ]:
learn = vision_learner(dls, resnet18, cbs=[NeptuneCallback(run=run, base_namespace="experiment_2")])

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.fit_one_cycle(2)

### Log model weights every n epochs

In [ ]:
n = 2
learn = vision_learner(
    dls,
    resnet18,
    metrics=accuracy,
    cbs=[
        SaveModelCallback(every_epoch=n),
        NeptuneCallback(run=run, base_namespace="experiment_3", upload_saved_models="all"),
    ],
)

In [ ]:
learn.fit_one_cycle(5)

### Pickling and logging the learner
Remove the NeptuneCallback class before pickling the learner object to avoid errors due to pickle's inability to pickle local objects (i.e., nested functions or methods).

In [ ]:
pickled_learner = "learner.pkl"
base_namespace = "experiment_4"
neptune_cbk = NeptuneCallback(run=run, base_namespace=base_namespace)
learn = vision_learner(
    dls,
    resnet18,
    metrics=accuracy,
    cbs=[neptune_cbk],
)
learn.fit_one_cycle(1)  # training
learn.remove_cb(neptune_cbk)  # remove NeptuneCallback
learn.export(f"./{pickled_learner}")  # export learner
run[f"{base_namespace}/pickled_learner"].upload(pickled_learner)  # (Neptune) upload pickled learner
learn.add_cb(neptune_cbk)  # add NeptuneCallback back again
learn.fit_one_cycle(1)  # continue training

### Log images

In [ ]:
batch = dls.one_batch()
for i, (x, y) in enumerate(dls.decode_batch(batch)):
    # Neptune supports torch tensors
    # fastai uses their own tensor type name TensorImage
    # so you have to convert it back to torch.Tensor
    run["images/one_batch"].append(
        File.as_image(x.as_subclass(torch.Tensor).permute(2, 1, 0) / 255.0),
        name=f"{i}",
        description=f"Label: {y}",
    )

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()